<a href="https://colab.research.google.com/github/MaiKhoa0101/MachineLearning/blob/main/synonym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask flask-ngrok sentence-transformers scikit-learn

In [ ]:
pip install pyngrok

In [ ]:
pip install flask-cors

In [ ]:
import json
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Tạo dictionary để tra cứu nhanh
gross_to_url = {}
for item in data:
    gross = item["gross"].lower().strip()
    if gross not in gross_to_url:
        gross_to_url[gross] = []
    gross_to_url[gross].append(item["url"])

texts = list(gross_to_url.keys())

# Load model - sử dụng model đa ngôn ngữ tốt hơn
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Pre-compute embeddings
print("🔄 Đang encode embeddings...")
embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
print(f"✅ Đã load {len(texts)} từ vựng")

app = Flask(__name__)

# Configuration
SIMILARITY_THRESHOLD = 0.80  # Ngưỡng 90%
# Cần thêm metadata về độ dài video vào JSON
def find_shortest_video(urls):
    # Logic để tìm video ngắn nhất
    return urls[0]  # Placeholder

def find_synonym(word, threshold=SIMILARITY_THRESHOLD):
    """
    Tìm từ đồng nghĩa cho một từ
    Returns: (synonym, accuracy, urls) hoặc None nếu không tìm thấy
    """
    word = word.lower().strip()

    # Kiểm tra exact match trước
    if word in gross_to_url:
        return (word, 100.0, gross_to_url[word])

    # Encode từ cần tìm
    word_embedding = model.encode([word])

    # Tính similarity
    similarities = cosine_similarity(word_embedding, embeddings)[0]

    # Tìm best match
    best_idx = np.argmax(similarities)
    best_score = similarities[best_idx]

    if best_score >= threshold:
        best_match = texts[best_idx]
        return (best_match, round(float(best_score * 100), 2), gross_to_url[best_match])

    return None

@app.route("/translate", methods=["POST"])
def translate():
    """
    API để dịch câu thành các từ chuẩn
    Input: {"sentence": "Tao xơi cơm rất ngon"}
    Output: Câu đã được chuẩn hóa và thông tin chi tiết
    """
    body = request.get_json()

    if not body or "sentence" not in body:
        return jsonify({"error": "Missing 'sentence' in request body"}), 400

    sentence = body["sentence"]
    words = sentence.split()

    result_words = []
    word_details = []
    skipped_words = []

    for word in words:
        synonym_result = find_synonym(word)

        if synonym_result:
            synonym, accuracy, urls = synonym_result
            result_words.append(synonym)
            word_details.append({
                "original": word,
                "synonym": synonym,
                "accuracy": accuracy,
                "urls": urls
            })
        else:
            # Giữ nguyên từ gốc nếu không tìm thấy
            result_words.append(word)
            skipped_words.append(word)

    return jsonify({
        "original_sentence": sentence,
        "translated_sentence": " ".join(result_words),
        "word_details": word_details,
        "skipped_words": skipped_words,
        "total_words": len(words),
        "translated_words": len(word_details),
        "skipped_count": len(skipped_words)
    })

@app.route("/search", methods=["POST"])
def search():
    """
    API để tìm từ đồng nghĩa cho nhiều từ
    Input: {"queries": ["tao", "xơi", "cơm"]}
    Output: Kết quả cho từng từ (CHỈ 1 URL TỐT NHẤT)
    """
    body = request.get_json()
    queries = body.get("queries", [])

    if isinstance(queries, str):
        queries = [queries]

    if not isinstance(queries, list) or len(queries) == 0:
        return jsonify({"error": "'queries' must be a non-empty array"}), 400

    results = {}

    for query in queries:
        synonym_result = find_synonym(query)

        if synonym_result:
            synonym, accuracy, urls = synonym_result

            # ✅ CHỈ LẤY URL ĐẦU TIÊN (hoặc random 1 cái)
            results[query] = {
                "found": True,
                "synonym": synonym,
                "accuracy": accuracy,
                "url": find_shortest_video(urls)
                # "all_urls": urls  # ❌ Không cần trả về tất cả
            }
        else:
            results[query] = {
                "found": False,
                "message": f"No synonym found with accuracy >= {SIMILARITY_THRESHOLD * 100}%"
            }

    return jsonify({
        "total_queries": len(queries),
        "threshold": SIMILARITY_THRESHOLD * 100,
        "results": results
    })

@app.route("/health", methods=["GET"])
def health():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "model": "paraphrase-multilingual-MiniLM-L12-v2",
        "vocabulary_size": len(texts),
        "threshold": SIMILARITY_THRESHOLD * 100
    })

if __name__ == '__main__':
    from pyngrok import ngrok

    # Expose URL công khai
    ngrok.set_auth_token("372B0cBmcE6XQkk7Y0JSSGdlw2P_3GqNZtELf3VTxdGDvaPMV")
    public_url = ngrok.connect(5500)

    print(f"\n{'='*60}")
    print(f"🚀 Flask server running at: {public_url}")
    print(f"{'='*60}\n")
    print(f"📊 Vocabulary size: {len(texts)}")
    print(f"🎯 Similarity threshold: {SIMILARITY_THRESHOLD * 100}%")
    print(f"\n📍 Endpoints:")
    print(f"  POST {public_url}/translate - Dịch câu")
    print(f"  POST {public_url}/search    - Tìm từ đồng nghĩa")
    print(f"  GET  {public_url}/health    - Health check")
    print(f"\n💡 Example request:")
    print(f'  curl -X POST {public_url}/translate \\')
    print(f'    -H "Content-Type: application/json" \\')
    print(f'    -d \'{{"sentence": "Tao xơi cơm rất ngon"}}\'')
    print(f"\n{'='*60}\n")

    app.run(host='0.0.0.0', port=5500, debug=False)

🔄 Đang encode embeddings...


Batches:   0%|          | 0/104 [00:00<?, ?it/s]

✅ Đã load 3318 từ vựng

🚀 Flask server running at: NgrokTunnel: "https://demoded-lourie-unpoulticed.ngrok-free.dev" -> "http://localhost:5500"

📊 Vocabulary size: 3318
🎯 Similarity threshold: 80.0%

📍 Endpoints:
  POST NgrokTunnel: "https://demoded-lourie-unpoulticed.ngrok-free.dev" -> "http://localhost:5500"/translate - Dịch câu
  POST NgrokTunnel: "https://demoded-lourie-unpoulticed.ngrok-free.dev" -> "http://localhost:5500"/search    - Tìm từ đồng nghĩa
  GET  NgrokTunnel: "https://demoded-lourie-unpoulticed.ngrok-free.dev" -> "http://localhost:5500"/health    - Health check

💡 Example request:
  curl -X POST NgrokTunnel: "https://demoded-lourie-unpoulticed.ngrok-free.dev" -> "http://localhost:5500"/translate \
    -H "Content-Type: application/json" \
    -d '{"sentence": "Tao xơi cơm rất ngon"}'


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5500
 * Running on http://172.28.0.12:5500
INFO:werkzeug:Press CTRL+C to quit


**Model chuyên tiếng việt**

In [ ]:
!pip install -q sentence-transformers transformers torch scikit-learn

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer(
    "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
)


In [ ]:
texts = [item["gross"] for item in data]

embeddings = model.encode(
    texts,
    show_progress_bar=True
)

print("✅ Đã tạo embedding tiếng Việt")


In [ ]:
def vietnamese_semantic_search(
    query,
    data,
    embeddings,
    threshold=0.8
):
    query_embedding = model.encode([query])

    similarities = cosine_similarity(
        query_embedding,
        embeddings
    )[0]

    results = []
    for item, score in zip(data, similarities):
        if score >= threshold:
            results.append({
                "gross": item["gross"],
                "url": item["url"],
                "accuracy": round(score * 100, 2)
            })

    results.sort(key=lambda x: x["accuracy"], reverse=True)
    return results


In [ ]:
query = "phụ huynh"

results = vietnamese_semantic_search(
    query=query,
    data=data,
    embeddings=embeddings
)

if not results:
    print("❌ Không có kết quả ≥ 80%")
else:
    print(f"Kết quả cho từ '{query}':\n")
    for r in results:
        print(f"{r['accuracy']}% → {r['gross']} → {r['url']}")


In [ ]:
import json
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# load data
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [item["gross"] for item in data]

#load model
model = SentenceTransformer(
    "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base"
)

embeddings = model.encode(texts)

app = Flask(__name__)

@app.route("/search", methods=["POST"])
def search():
    body = request.get_json()

    if not body or "query" not in body:
        return jsonify({
            "error": "Missing 'query' in request body"
        }), 400

    query = body["query"]
    query_embedding = model.encode([query])

    similarities = cosine_similarity(
        query_embedding,
        embeddings
    )[0]

    # Tìm index của kết quả có độ tương đồng cao nhất
    best_match_index = similarities.argmax()
    best_score = similarities[best_match_index]

    # Chỉ trả về best match nếu độ chính xác >= 90%
    if best_score >= 0.85:
        best_match = data[best_match_index]
        result = {
            "gross": best_match["gross"],
            "url": best_match["url"],
            "accuracy": round(float(best_score * 100), 2)
        }

        return jsonify({
            "query": query,
            "result": [result]
        })
    else:
        return jsonify({
            "query": query,
            "result": [],
            "message": "No match found with accuracy >= 85%"
        })

if __name__ == '__main__':
    # Expose URL công khai
    from flask import Flask
    from pyngrok import ngrok

    ngrok.set_auth_token("372B0cBmcE6XQkk7Y0JSSGdlw2P_3GqNZtELf3VTxdGDvaPMV")
    public_url = ngrok.connect(5500)
    print(f"Flask server running at: {public_url}")

    app.run(host='0.0.0.0', port=5500, debug=False)

# Mục mới

In [ ]:
import json
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [item["gross"] for item in data]

# Load model
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Pre-compute embeddings for all data
print("Encoding data embeddings...")
embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
print(f"Loaded {len(data)} items")

app = Flask(__name__)

# Configuration
SIMILARITY_THRESHOLD = 0.80  # Ngưỡng độ tương đồng tối thiểu
MAX_RESULTS = 100  # Số lượng kết quả tối đa trả về
BATCH_SIZE = 32  # Batch size cho encoding

@app.route("/search", methods=["POST"])
def search():
    body = request.get_json()
    queries = body.get("query", [])
    max_per_query = body.get("max_results_per_query", 1)  # Mặc định 3

    if isinstance(queries, str):
        queries = [queries]

    if not isinstance(queries, list) or len(queries) == 0:
        return jsonify({"error": "'query' must be a non-empty array"}), 400

    # Loại bỏ duplicate
    unique_queries = list(set(queries))

    # Batch encode
    query_embeddings = model.encode(unique_queries, batch_size=32)

    # Tính similarity
    similarities = cosine_similarity(query_embeddings, embeddings)

    # Thu thập kết quả với giới hạn per query
    results_by_query = {}

    for query_idx, query in enumerate(unique_queries):
        query_results = []

        # Lấy indices sorted theo similarity (cao -> thấp)
        sorted_indices = np.argsort(similarities[query_idx])[::-1]

        for item_idx in sorted_indices:
            score = similarities[query_idx][item_idx]

            if score >= 0.8:  # Threshold
                query_results.append({
                    "gross": data[item_idx]["gross"],
                    "url": data[item_idx]["url"],
                    "accuracy": round(float(score * 100), 2)
                })

                # Giới hạn số kết quả cho mỗi query
                if len(query_results) >= max_per_query:
                    break

        results_by_query[query] = query_results

    return jsonify({
        "total_queries": len(unique_queries),
        "max_results_per_query": max_per_query,
        "results": results_by_query
    })


@app.route("/health", methods=["GET"])
def health():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "model": "paraphrase-multilingual-MiniLM-L12-v2",
        "data_items": len(data),
        "threshold": SIMILARITY_THRESHOLD,
        "max_results": MAX_RESULTS
    })


@app.route("/config", methods=["GET"])
def get_config():
    """Get current configuration"""
    return jsonify({
        "similarity_threshold": SIMILARITY_THRESHOLD,
        "max_results": MAX_RESULTS,
        "batch_size": BATCH_SIZE,
        "total_data_items": len(data)
    })


if __name__ == '__main__':
    # Expose URL công khai với ngrok
    from pyngrok import ngrok

    ngrok.set_auth_token("372B0cBmcE6XQkk7Y0JSSGdlw2P_3GqNZtELf3VTxdGDvaPMV")
    public_url = ngrok.connect(5500)
    print(f"\n{'='*60}")
    print(f"🚀 Flask server running at: {public_url}")
    print(f"{'='*60}\n")
    print(f"📊 Loaded {len(data)} items")
    print(f"🎯 Similarity threshold: {SIMILARITY_THRESHOLD * 100}%")
    print(f"📋 Max results: {MAX_RESULTS}")
    print(f"\nEndpoints:")
    print(f"  POST {public_url}/search - Search for synonyms")
    print(f"  GET  {public_url}/health - Health check")
    print(f"  GET  {public_url}/config - View configuration")
    print(f"\n{'='*60}\n")

    app.run(host='0.0.0.0', port=5500, debug=False)

In [ ]:
import json

with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"✅ Đã load {len(data)} bản ghi")


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer(
    "paraphrase-multilingual-MiniLM-L12-v2"
)


In [ ]:
texts = [item["gross"] for item in data]

embeddings = model.encode(
    texts,
    show_progress_bar=True
)

print("✅ Đã tạo embedding")


In [ ]:
def semantic_search_80_percent(
    query,
    data,
    embeddings
):
    query_embedding = model.encode([query])

    similarities = cosine_similarity(
        query_embedding,
        embeddings
    )[0]

    results = []
    for item, score in zip(data, similarities):
        if score >= 0.9:   # 🔥 CHỈ LẤY >= 80%
            results.append({
                "gross": item["gross"],
                "url": item["url"],
                "accuracy": round(float(score * 100), 2)
            })

    results.sort(key=lambda x: x["accuracy"], reverse=True)
    return results


In [ ]:
query = "ba mẹ"

results = semantic_search_80_percent(
    query=query,
    data=data,
    embeddings=embeddings
)

if not results:
    print("❌ Không có kết quả >= 80%")
else:
    print(f"Kết quả cho từ '{query}':\n")
    for r in results:
        print(f"{r['accuracy']}% → {r['gross']} → {r['url']}")
